In [4]:
import gmsh
import sys
import os
import numpy as np
import import_ipynb
import matplotlib.pyplot as plt
from fem_pre_processing import create_domain, read_mesh
from fem_processing import master_domain, gaussian_quadrature, matrices_assembly as assembly
from fem_pos_processing import graph_results

ModuleNotFoundError: No module named 'fem_pre_processing'

# `set_analytical_solution()`

In [ ]:
def set_analytical_solution(x, y):
    """
    Calculate the vector solution u(x, y) and its gradient ∇u(x, y) in a 2D matrix form.

    Parameters:
    x (float or array-like): x-coordinate(s)
    y (float or array-like): y-coordinate(s)
    
    Returns:
    tuple:
        - u (ndarray): Vector solution u(x, y) as a 2D column matrix:
            [[ cos(πx) sin(πy)],
             [-sin(πx) cos(πy)]]

        - grad_u (ndarray): Gradient ∇u(x, y), a 2x2 matrix with:
            [[∂u1/∂x, ∂u1/∂y],
             [∂u2/∂x, ∂u2/∂y]]
    """
    cos_pix = np.cos(np.pi * x)
    sin_pix = np.sin(np.pi * x)
    cos_piy = np.cos(np.pi * y)
    sin_piy = np.sin(np.pi * y)

    # Vector solution (2x1 matrix)
    u = np.array([[ cos_pix * sin_piy],
                  [-sin_pix * cos_piy]])
    
    # Função forçante (fonte)
    f = (2 * np.pi ** 2 + 1) * u
    
    # Gradient of u (Jacobian matrix, 2x2)
    grad_u = np.array([[ np.pi * (np.cos(2 * np.pi * x) - 0.5),  0],
                       [-np.pi * np.cos(np.pi * x) * np.cos(np.pi * y),
                         np.pi * np.sin(np.pi * x) * np.sin(np.pi * y)]])
    
    return u, f, grad_u

# `get_edge_key_map()`

In [ ]:
def get_edge_key_map(mesh_data):
    # Construir um dicionário auxiliar para mapear arestas diretamente às suas chaves
    edge_key_map = {}
    for key, edge in mesh_data['edges'].items():
        # Usamos uma tupla ordenada para representar a aresta, garantindo unicidade independente da direção
        edge_nodes = tuple(sorted(edge['conn']))
        edge_key_map[edge_nodes] = key

    return edge_key_map

# `apply_physics()`

In [ ]:
def apply_physics(FINITE_ELEMENT, mesh_data):
    """
    Adiciona uma nova chave 'source' a cada dicionário em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - element_type: Tuple (tipo do elemento, ordem).
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """
    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']
    edges_data = mesh_data['edges']

    # Mapeamento das arestas diretamente às suas chaves    
    edge_key_map = get_edge_key_map(mesh_data)

    # Atualiza dados físicos dos elementos
    for key, cell in cell_data.items():  
        # Adicionar as propriedades do materiais ao dicionário da célula
        cell['stiffness_term'] = []
        cell['mass_term'] = []
        cell['source'] = []

        # Geometric data    
        xc = np.mean([mesh_data['nodes'][node]['xg'][0] for node in cell['conn']])
        yc = np.mean([mesh_data['nodes'][node]['xg'][1] for node in cell['conn']])
        Je = assembly.vectorial_jacobian(mesh_data, cell)
        area = np.abs(np.linalg.det(Je)) * 0.5 

        cell['geo']['centroid'] = (xc, yc)
        cell['geo']['area'] = area
        
        # Calcular o valor de 'a' de acordo com a física do problema
        # mur = cell['material']['relative_magnetic_permeability']
        # er = cell['material']['relative_electric_permittivity']
        a = cell['material']['a_constant']
        
        # Get the global coordinates of the nodes
        ai = [nodes_data[idx]['xg'] for idx in cell['conn']]

        # Adicionar a nova chave 'material' ao dicionário da célula
        gauss_points, _ = gaussian_quadrature.gauss_data(FINITE_ELEMENT)
        for xik in gauss_points: 
            xg = assembly.isomapping_to_global_coordinates(FINITE_ELEMENT, ai, xik)
            cell['stiffness_term'].append(np.eye(2))
            cell['mass_term'].append(a)           
            cell['source'].append(set_analytical_solution(*xg)[1])

        # Contorno da fronteira
        gamma_a = {idx: node for idx, node in enumerate(cell['conn'])
                        if nodes_data[node]['bc']['tag'] == 101}  
        if len(gamma_a) > 1:
            cell_data[key]['contour'] = {'type': 'Dirichlet', 'conn_contour': gamma_a}

        # Atualiza a conectividade de arestas da célula
        cell['conn_edge'] = [
            edge_key_map[tuple(sorted([cell['conn_sorted'][0], cell['conn_sorted'][1]]))],  # e1: nó 1 -> nó 2
            edge_key_map[tuple(sorted([cell['conn_sorted'][0], cell['conn_sorted'][2]]))],  # e2: nó 1 -> nó 3
            edge_key_map[tuple(sorted([cell['conn_sorted'][1], cell['conn_sorted'][2]]))]   # e3: nó 2 -> nó 3
        ]

        

    # Atualiza dados físicos das arestas
    for key, edge in edges_data.items():
        # Nós da aresta
        n0, n1 = edge['conn'][0], edge['conn'][1]
        
        # Adiciona os potenciais de Dirichlet sobre as arestas
        if nodes_data[n0]['bc']['tag'] == 101 and nodes_data[n1]['bc']['tag'] == 101:
            edge['bc'] = {
                'tag': 101,
                'type': 'Dirichlet',
                'value': nodes_data[n0]['bc']['value'],
                'name': 'contour_domain'}
            
        # Coordenadas dos nós inicial e final
        xe_0, ye_0 = mesh_data['nodes'][n0]['xg']
        xe_1, ye_1 = mesh_data['nodes'][n1]['xg']
    
        # Adiciona o tamanho da aresta
        edge['len'] = ((xe_1 - xe_0)**2 + (ye_1 - ye_0)**2) ** 0.5

    return mesh_data

# `create_meshed_domain()`

In [ ]:
def create_meshed_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, auto_save=True, view_mesh=False):
    type, order = FINITE_ELEMENT
    mesh_data = {}
    vertices = [(-1, -1, 0), (-1, 1, 0), (1, 1, 0), (1, -1, 0)]

    # Define a new model
    gmsh.initialize()
    gmsh.model.add("vectorial_poisson")

    # Define the points of the domain.
    point_tags = create_domain.add_points(vertices, h)

    # Criar linhas para formar as bordas do quadrado
    line_tags = create_domain.add_lines(point_tags)
    
    # Criar um loop de linha e uma superfície plana
    free_space = create_domain.add_surface(line_tags)

    # Adicionar grupos físicos para Dim=1
    for i, tag in enumerate([line_tags]):
        gmsh.model.addPhysicalGroup(1, tags=tag, tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

    # Adicionar grupos físicos para Dim=2
    gmsh.model.addPhysicalGroup(2, [free_space], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name'])  
    
    # Gerar a malha 2D
    gmsh.option.setNumber("Mesh.SaveAll", 1)
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(order)

    # Visualizar a malha no ambiente Gmsh (opcional)
    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/vectorial_poisson_domain_{type}{order}.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)
        read_mesh.basic_info()
    
    # Create mesh Structure Data from gmsh
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['edges'] = read_mesh.get_edge_data()

    # Apply physics to the problem
    mesh_data = apply_physics(FINITE_ELEMENT, mesh_data)

    # Finalize the gmsh model e return the mesh data
    gmsh.finalize()
    return mesh_data        

# `curl_u_at_domain()`

In [ ]:
def curl_u_at_domain(mesh_data):
    curl_u = {}
    for key, edge in mesh_data['edges'].items():
        # Nós da aresta
        n0, n1 = edge['conn'][0], edge['conn'][1]

        # Coordenadas dos nós inicial e final
        xe_0, ye_0 = mesh_data['nodes'][n0]['xg']
        xe_1, ye_1 = mesh_data['nodes'][n1]['xg']
        
        # Ponto médio da aresta
        xmid, ymid = (xe_0 + xe_1) / 2, (ye_0 + ye_1) / 2
        
        # Vetor tangente à aresta
        tangent_vector = np.array([[xe_1 - xe_0], [ye_1 - ye_0]])

        # Campo vetorial no ponto médio da aresta, u(x_mid, y_mid) 
        u_vector = set_analytical_solution(xmid, ymid)[0]

        # Produto escalar entre o campo vetorial e o vetor tangente
        ## O método .item() extrai o valor escalar de um array de tamanho 1x1
        curl_u[key] = (u_vector.T @ tangent_vector).item()

    # Ordenar o dicionário de acordo com as chaves
    return curl_u

# `interpolate_solution()`

In [ ]:
def interpolate_solution(mesh_data, curl_uh, xik_master):
    # Dicionário para armazenar os potenciais vetoriais interpolados em cada célula
    uh_at_cell = {}

    # Percorre cada célula da malha
    for key, cell in mesh_data['cell'].items():    
        Je = assembly.vectorial_jacobian(mesh_data, cell)    
        JinvT = np.linalg.inv(Je).T
        phi_hat = master_domain.shape_functions_n0(*xik_master)
        uh_at_cell[key] = 0

        # Interpolação dos potenciais vetoriais em cada célula
        for i, edge in enumerate(cell['conn_edge']):
            uh_at_cell[key] += curl_uh[edge] * (JinvT @ phi_hat[i])
    
    return uh_at_cell

# `plot_shape_functions()`

In [ ]:
def plot_shape_functions_n0():
    """
    Plota as funções de forma vetoriais de Nedelec do tipo 1 no triângulo de referência.
    """
    # Definir uma grade de pontos dentro do triângulo de referência
    xi = np.linspace(0, 1, 10)
    eta = np.linspace(0, 1, 10)
    xi_grid, eta_grid = np.meshgrid(xi, eta)
    
    # Filtrar pontos dentro do triângulo de referência
    inside_triangle = xi_grid + eta_grid <= 1
    xi_grid = xi_grid[inside_triangle]
    eta_grid = eta_grid[inside_triangle]

    # Criar uma figura
    plt.figure(figsize=(12, 6))

    # Lista de cores
    colors = ['blue', 'red', 'green']
    
    # Iterar sobre cada função de forma
    for idx in range(3):
        ax = plt.subplot(1, 3, idx + 1)  # Subplots para cada função de forma
        
        # Inicializar vetores para quiver
        u = np.zeros_like(xi_grid)
        v = np.zeros_like(eta_grid)
        
        # Calcular os vetores das funções de forma
        for i, (xi, eta) in enumerate(zip(xi_grid, eta_grid)):
            N = master_domain.shape_functions_n0(xi, eta)
            u[i], v[i] = N[idx][0, 0], N[idx][1, 0]
        
        # Plotar vetores com quiver
        plt.quiver(xi_grid, eta_grid, u, v,
                    angles='xy', scale_units='xy', scale=10, color=colors[idx]
        )
        
        # Plotar o triângulo de referência
        plt.plot([0, 1, 0, 0], [0, 0, 1, 0], 'k-')
        plt.title(fr'$\hat{{\varphi}}_{{{idx+1}}}$')
        plt.xlabel(r'$\xi$')
        plt.ylabel(r'$\eta$')
        plt.xlim(-0.25, 1.25)
        plt.ylim(-0.25, 1.25)
        # plt.axis('equal')
        plt.tight_layout()

        # # Remover os contornos (spines) do gráfico
        # for spine in ax.spines.values():
        #     spine.set_visible(False)
    
    # Salvando o arquivo no formato SVG
    filepath = graph_results.get_dir(f"pre_processing/pictures/nedelec_shape_functions_n0.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

# `plot_analytical_solution()`

In [ ]:
def plot_analytical_solution():
    """
    Plot the exact vector solution u(x, y) using quiver() over the domain [-1, 1]².
    The solution is defined as:
        u1(x, y) = cos(πx)sin(πx)
        u2(x, y) = -sin(πx)cos(πy)
    """
    # Define the exact solution
    x, y = np.meshgrid(np.linspace(-1, 1, 30), np.linspace(-1, 1, 30))

    # Compute the exact solution components
    u1 = np.cos(np.pi * x) * np.sin(np.pi * y)  # First component of u
    u2 = -np.sin(np.pi * x) * np.cos(np.pi * y)  # Second component of u

    # Components of source function f(x, y) = (2π² + 1)u(x, y)
    src1, src2 = (2*np.pi**2+1) * u1, (2*np.pi**2+1) * u2   

    # Plot the vector field using quiver()
    plt.figure(figsize=(8, 8))
    plt.quiver(x, y, u1, u2, color='b', angles='xy', scale_units='xy', scale=15, width=0.003)
    plt.title("Exact Solution $\\mathbf{u}(x, y)$ over $[-1, 1]^2$")
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.xlim([-1, 1])
    plt.ylim([-1, 1])
    plt.axis('equal')
    plt.tight_layout()
    
    # Salvando o arquivo no formato SVG
    filepath = graph_results.get_dir(f"pos_processing/pictures/analytical_solution_at_omegac.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

    # Plot the vector field using quiver()
    plt.figure(figsize=(8, 8))
    plt.quiver(x, y, src1, src2, color='g', angles='xy', scale_units='xy', scale=300, width=0.003)
    plt.title("Source $\\mathbf{f}(x, y)$ over $[-1, 1]^2$")
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.xlim([-1, 1])
    plt.ylim([-1, 1])
    plt.axis('equal')
    plt.tight_layout()
    
    # Salvando o arquivo no formato SVG
    filepath = graph_results.get_dir(f"pos_processing/pictures/source_at_omegac.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

In [ ]:
def plot_analytical_solution_old():
    """
    Plot the exact vector solution u(x, y) and the source function f(x, y) using quiver() over the domain [-1, 1]².
    
    The solution is defined as:
        u1(x, y) = cos(πx)sin(πy)
        u2(x, y) = -sin(πx)cos(πy)
    
    The source function is given by:
        f(x, y) = (2π² + 1)u(x, y)
    """
    # Define the grid over the domain [-1, 1]²
    x, y = np.meshgrid(np.linspace(-1, 1, 30), np.linspace(-1, 1, 30))

    # Compute the exact solution and the source function
    u, f, _ = set_analytical_solution(x, y)  # Unpack u and f from the function

    # Extract components of the solution and source function
    u1, u2 = u[0], u[1]
    src1, src2 = f[0], f[1]
    
    # Plot the exact solution vector field
    plt.figure(figsize=(8, 8))
    plt.quiver(x, y, u1, u2, color='b', angles='xy', scale_units='xy', scale=15, width=0.003)
    plt.title("Exact Solution $\\mathbf{u}(x, y)$ over $[-1, 1]^2$")
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.xlim([-1, 1])
    plt.ylim([-1, 1])
    plt.axis('equal')
    plt.tight_layout()
    
    # Save the plot as an SVG file
    filepath = graph_results.get_dir("pos_processing/pictures/analytical_solution_at_omegac.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

    # Plot the source function vector field
    plt.figure(figsize=(8, 8))
    plt.quiver(x, y, src1, src2, color='g', angles='xy', scale_units='xy', scale=300, width=0.003)
    plt.title("Source $\\mathbf{f}(x, y)$ over $[-1, 1]^2$")
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.xlim([-1, 1])
    plt.ylim([-1, 1])
    plt.axis('equal')
    plt.tight_layout()
    
    # Save the source function plot as an SVG file
    filepath = graph_results.get_dir("pos_processing/pictures/source_at_omegac.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")
